In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GenerationConfig, GPT2Config
import torch
import torch.nn as nn
import os
from pathlib import Path

import transformers
from onnxruntime import InferenceSession
from transformers.onnx.features import FeaturesManager
from optimum.exporters.tasks import TasksManager

In [2]:
import numpy as np

def save_to(x, fp):
    os.makedirs(os.path.dirname(fp), exist_ok=True)
    print(x.dtype)
    np_x = x.detach().numpy()
    with open(fp, 'wb') as f:
        np_x.tofile(f)

def save_past_key_values(x, n, d):
    for l in range(n):
        k, v = x[l]
        fp = os.path.join(d, "past_key_values", "{}.{}".format('past', l))
        save_to(k, fp + '.key')
        save_to(v, fp + '.value')

In [3]:
torch.manual_seed(2333)
# torch.set_num_threads(1)

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model.eval()
# model.config.return_dict = False
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input

{'input_ids': tensor([[3041, 5372,  502,  416,  597, 2420,  345, 1549,  588,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
model.transformer.wte = nn.Embedding(50264, 1024)
model.lm_head = nn.Linear(1024, 50264, bias=False)
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50264, bias=False)
)


In [7]:
print(model.transformer.h[0])

GPT2Block(
  (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


In [8]:
# Stage 0
encoded_input_0 = {
    'input_ids': torch.randint(0, 50264, (64, 255)),
    'past_key_values': None
}


In [9]:
%%time
with torch.no_grad():
    output = model(**encoded_input_0)
print(output.logits.shape)
print(output.past_key_values[0][0].size())

torch.Size([64, 255, 50264])
torch.Size([64, 16, 255, 64])
CPU times: user 6min 42s, sys: 34.5 s, total: 7min 16s
Wall time: 27.7 s


In [10]:
# prepare for stage 1
next_input = torch.argmax(output.logits[..., -1, :], dim=-1, keepdim=True)
next_input.size()

torch.Size([64, 1])

In [12]:
%%time
# Stage 1 without current past return
model.config.use_cache = False

with torch.no_grad():
    next_output = model(next_input, past_key_values=output.past_key_values)
print(next_output)

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ 1.5192,  0.9893, -2.6038,  ...,  7.5997,  4.8440,  9.3179]],

        [[ 1.3508, -0.0699, -0.2945,  ...,  2.4565,  1.1106,  1.6794]],

        [[ 4.3459,  2.4910, -2.1035,  ...,  5.9881,  6.9970,  8.0259]],

        ...,

        [[ 2.8136,  1.4874, -1.1700,  ...,  5.1903,  4.6788,  6.8327]],

        [[ 1.1831,  0.5177, -0.7947,  ...,  3.9255,  2.6822,  4.2822]],

        [[ 1.9604,  0.6672, -0.4625,  ...,  3.1965,  2.2869,  2.8252]]]), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)
CPU times: user 7.73 s, sys: 1.07 s, total: 8.8 s
Wall time: 559 ms


In [13]:
# Export
def past_key_values_names(n: int, prefix: str):
    for i in range(n):
        yield "{}.{}.key".format(prefix, i)
        yield "{}.{}.value".format(prefix, i)
input_names = ["input_ids"] + list(past_key_values_names(24, "past"))
# output_names = ["logits"] + list(past_key_values_names(24, "present"))
output_names = ["logits"]
print(input_names)
print(output_names)

['input_ids', 'past.0.key', 'past.0.value', 'past.1.key', 'past.1.value', 'past.2.key', 'past.2.value', 'past.3.key', 'past.3.value', 'past.4.key', 'past.4.value', 'past.5.key', 'past.5.value', 'past.6.key', 'past.6.value', 'past.7.key', 'past.7.value', 'past.8.key', 'past.8.value', 'past.9.key', 'past.9.value', 'past.10.key', 'past.10.value', 'past.11.key', 'past.11.value', 'past.12.key', 'past.12.value', 'past.13.key', 'past.13.value', 'past.14.key', 'past.14.value', 'past.15.key', 'past.15.value', 'past.16.key', 'past.16.value', 'past.17.key', 'past.17.value', 'past.18.key', 'past.18.value', 'past.19.key', 'past.19.value', 'past.20.key', 'past.20.value', 'past.21.key', 'past.21.value', 'past.22.key', 'past.22.value', 'past.23.key', 'past.23.value']
['logits']


In [14]:
dummy_input = {
    'input_ids': next_input,
    'past_key_values': output.past_key_values
}
torch.onnx.export(model,
                  dummy_input,
                  "./model_onnx/gpt2.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [15]:
save_to(dummy_input['input_ids'], './model_onnx/input_ids.dat')
save_past_key_values(dummy_input['past_key_values'], 24, './model_onnx')

torch.int64
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
